#gureBERT

In [0]:
# gureBERT

!git clone --recursive  https://github.com/zmwebdev/gureBERT
%cd gureBERT

!pip install sentencepiece
!install -d spModels
# eu
!python src/sentence-split.py --config eu.config.ini --do_lower_case 
!python src/train-sentencepiece.py --config eu.config.ini

!head -n 100 spModels/eu.vocab

# en-eu
!python src/sentence-split.py --config en-eu.config.ini --do_lower_case 
!python src/train-sentencepiece.py --config en-eu.config.ini

!head -n 100 spModels/en-eu.vocab


In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
# google bucket
!gsutil cp -r gs://gurebert/gureBERT/spModels .

In [40]:
import sys
sys.path.append("src")

import tokenization_sentencepiece as tokenization

tokenizer = tokenization.FullTokenizer(
    model_file="spModels/eu.model",
    vocab_file="spModels/eu.vocab",
    do_lower_case=True)

text1 = "Nere kotxea aitonaren etxe alboan dago, bere kolorea gorria da."

print(tokenizer.tokenize(text1))

Loaded a trained SentencePiece model.
['▁ne', 're', '▁', 'ko', 'txea', '▁ai', 'ton', 'aren', '▁etxe', '▁alb', 'o', 'an', '▁dago', ',', '▁bere', '▁kolore', 'a', '▁gorria', '▁da', '.']


In [13]:
tokenizer_en_eu = tokenization.FullTokenizer(
    model_file="spModels/en-eu.model",
    vocab_file="spModels/en-eu.vocab",
    do_lower_case=True)

text1 = "Nere kotxea aitonaren etxe alboan dago, bere kolorea gorria da."

print(tokenizer_en_eu.tokenize(text1))

Loaded a trained SentencePiece model.
['▁ne', 're', '▁ko', 'txea', '▁a', 'it', 'on', 'aren', '▁etxe', '▁', 'albo', 'an', '▁dago', ',', '▁bere', '▁kolore', 'a', '▁gorria', '▁da', '.']


In [14]:
text1 = "The Italian cities of Milan and Cortina d'Ampezzo are chosen as the joint hosts of the 2026 Winter Olympics and Winter Paralympics."

print("EN-EU: {}".format(tokenizer_en_eu.tokenize(text1)))
print("EU: {}".format(tokenizer.tokenize(text1)))

print("EN-EU: {}".format(len(tokenizer_en_eu.tokenize(text1))))
print("EU: {}".format(len(tokenizer.tokenize(text1))))

EN-EU: ['▁the', '▁italian', '▁cities', '▁of', '▁milan', '▁and', '▁cort', 'ina', '▁d', "'", 'amp', 'e', 'zzo', '▁are', '▁chosen', '▁as', '▁the', '▁joint', '▁hosts', '▁of', '▁the', '▁20', '26', '▁winter', '▁olympics', '▁and', '▁winter', '▁para', 'lympic', 's', '.']
EU: ['▁the', '▁italia', 'n', '▁cit', 'ies', '▁of', '▁milan', '▁and', '▁cort', 'ina', '▁d', "'", 'amp', 'ez', 'zo', '▁are', '▁ch', 'os', 'en', '▁as', '▁the', '▁jo', 'in', 't', '▁host', 's', '▁of', '▁the', '▁20', '26', '▁win', 'ter', '▁', 'oly', 'mp', 'ics', '▁and', '▁win', 'ter', '▁para', 'ly', 'mp', 'ics', '.']
EN-EU: 31
EU: 44


In [0]:
# Pre-Training
# https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/pretraining.ipynb

import tensorflow as tf

from google.colab import auth
auth.authenticate_user()

#Check TPU devices

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

In [0]:
GS = 'gs://gurebert/gureBERT'
  
#!gsutil cp -r spModels $GS/
#!gsutil cp -r corpus $GS/

## EU

In [0]:
!python3 src/create_pretraining_data.py \
    --input_file=$GS/corpus/eu/2014wiki.eu.sent_splited \
    --output_file=$GS/pretraining.tf.data \
    --model_file=spModels/eu.model \
    --vocab_file=spModels/eu.vocab \
    --do_lower_case=True

In [0]:
#!install -d gureBERT

!python src/run_pretraining.py \
  --config_file eu.config.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT/test \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=256 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=100 \
  --num_warmup_steps=10 \
  --save_checkpoints_steps=100 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
#  --num_train_steps=1000000 \

# num_train_steps ? zenbatekoa?
# 

In [0]:
#pretraining jarraipena 

GS = 'gs://gurebert/gureBERT'

!python src/run_pretraining.py \
  --config_file eu.config.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT/test \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=200 \
  --num_warmup_steps=10 \
  --save_checkpoints_steps=200 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --init_checkpoint=$GS/eu.gureBERT/test \
#  --num_train_steps=1000000 \


## EN-EU

In [0]:
FILES = "./corpus/en-eu/2014wiki.eu.sent_splited,./corpus/en-eu/2019wiki-10k.en.sent_splited"

!python3 src/create_pretraining_data.py \
    --input_file={FILES} \
    --output_file=$GS/pretraining-en_eu.tf.data \
    --model_file=spModels/en-eu.model \
    --vocab_file=spModels/en-eu.vocab \
    --do_lower_case=True

In [0]:
!python src/run_pretraining.py \
  --config_file en-eu.config.ini \
  --input_file=$GS/pretraining-en_eu.tf.data \
  --output_dir=$GS/en-eu.gureBERT \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --init_checkpoint=$GS/en-eu.gureBERT/ \
  #--num_train_steps=1000000 \


## SQuAD

In [0]:
# pre training SQuAD

# https://github.com/google-research/bert#squad-20

#!git clone https://github.com/zmwebdev/bert.git
#%cd bert

!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
# evaluation script: download file from a url that returns a save dialog box : https://superuser.com/questions/795265/download-file-from-a-url-that-returns-a-save-dialog-box#795269
!wget --content-disposition https://worksheets.codalab.org/rest/bundles/0x6b567e1cf2e041ec80d7098f031c5c9e/contents/blob/


In [21]:
# sentenpiece tokenizer erabili behar da!!: src/run_squad.py

!gsutil cp -r gs://gurebert/gureBERT/spModels .

 
!python src/run_squad.py \
  --vocab_file=./spModels/en-eu.vocab \
  --model_file=./spModels/en-eu.model \
  --bert_config_file=bert_config.json \
  --do_lower_case=True \
  --do_train=True \
  --train_file=train-v2.0.json \
  --do_predict=True \
  --predict_file=dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=0.1 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=gs://gurebert/gureBERT/squad/ \
  --use_tpu=True \
  --tpu_name=$TPU_NAME \
  --version_2_with_negative=True \
  --init_checkpoint=gs://gurebert/gureBERT/squad/ \
    

Copying gs://gurebert/gureBERT/spModels/en-eu.model...
Copying gs://gurebert/gureBERT/spModels/en-eu.vocab...
Copying gs://gurebert/gureBERT/spModels/eu.model...
Copying gs://gurebert/gureBERT/spModels/eu.vocab...
- [4 files][  2.5 MiB/  2.5 MiB]                                                
Operation completed over 4 objects/2.5 MiB.                                      
W0705 20:55:38.556497 139929338054528 deprecation_wrapper.py:119] From /content/gureBERT/src/../bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0705 20:55:38.558534 139929338054528 deprecation_wrapper.py:119] From src/run_squad.py:1293: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0705 20:55:38.559260 139929338054528 deprecation_wrapper.py:119] From src/run_squad.py:1137: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0705 20:55:38.559424 139929338054528 d

## wordpiece erabiliz

- https://towardsdatascience.com/pre-training-bert-from-scratch-with-cloud-tpu-6e2f71028379
- https://int-deepset-models-bert.s3.eu-central-1.amazonaws.com/pytorch/bert-base-german-cased-vocab.txt




### EU

In [5]:
import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('./spModels/eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

Learnt vocab size: 17217
Sample tokens: ['ong', '▁tourra', '911', '▁zelako', 'oinu', '▁160', '▁andrea', 'atzeko', '▁god', 'atxi']


In [0]:
# hau errepasatu. adibidez @ ez dut ondo egiten... [UNK] agertzen da...
# adibidez, 

#import string
#def parse_sentencepiece_token(token):
#    if token.startswith("▁"):
#        return token[1:]
#    else:
#        if token in string.punctuation:
#            return token
#        else:
#            return "##" + token
          

def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token          

In [0]:
import string

bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

VOC_FNAME2 = "vocab2.txt"
with open(VOC_FNAME2, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")


In [42]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME2)
text1 = "Nere kotxea aitonaren etxe alboan dago!, eta bere kolorea gorria da. ni@ni.eus erabili hitzhauezdaexixtitzen !~gg _*] "
print(bert_tokenizer.tokenize(text1))
print(tokenizer.tokenize(text1))

# [UNK] ak agertzen dira adibidez \koma, @, ..-rekin bert wordpiece erabiltzean
# BERT tokenizer-ak _run_split_on_puntuaction egiten du: kaixo, -> [kaixo] [,] ; 

['ne', '##re', 'ko', '##txea', 'ait', '##onar', '##en', 'etxe', 'alb', '##oan', 'dago', '!', '[UNK]', 'eta', 'bere', 'kolore', '##a', 'gorria', 'da', '[UNK]', 'ni', '[UNK]', 'ni', '[UNK]', 'eus', 'erabili', 'hitz', '##haue', '##z', '##da', '##e', '##xix', '##titz', '##en', '!', '[UNK]', 'g', '##g', '[UNK]', '*', '[UNK]']
['▁ne', 're', '▁', 'ko', 'txea', '▁ai', 'ton', 'aren', '▁etxe', '▁alb', 'o', 'an', '▁dago', '!', ',', '▁eta', '▁bere', '▁kolore', 'a', '▁gorria', '▁da', '.', '▁ni', '@', 'ni', '.', 'eus', '▁erabili', '▁hitz', 'hau', 'ez', 'da', 'e', 'xix', 'ti', 'tzen', '▁!', '~', 'gg', '▁', '_', '*', ']']


In [30]:
import string

bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
# puntuazio sinboloak gehitu 
ctrl_symbols_end = list(string.punctuation)
#ctrl_symbols = ["[UNK]"]
bert_vocab = ctrl_symbols + bert_vocab + ctrl_symbols_end

#bert_vocab += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]
print(len(bert_vocab))

45036


In [0]:
VOC_FNAME = "vocab.txt"
with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [41]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
#text1 = "Nere kotxea aitonaren etxe alboan dago!, eta bere kolorea gorria da. ni@ni.eus erabili hitzhauezdaexixtitzen !~gg _*] "
#text1 = "44.579.000 km²ko eremua eta 4.140.336.501 biztanle ditu"
#text1 = "biologia izena grezierazko bi osagai erabiliz eratua da: βίος, bios, «bizitza»; eta -λογία, -logia, «azterketa»."
text1 = "手洗い"
print("num:{}, tokenak:{}".format(len(bert_tokenizer.tokenize(text1)),bert_tokenizer.tokenize(text1)))
print("num:{}, tokenak:{}".format(len(tokenizer.tokenize(text1)),tokenizer.tokenize(text1)))

num:3, tokenak:['[UNK]', '[UNK]', '[UNK]']
num:2, tokenak:['▁', '手洗い']


In [37]:
# 
def bert_tokenize_corpus(filepath):
  tokens = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      token_line = bert_tokenizer.tokenize(line)
      tokens += token_line
      if '[UNK]' in token_line:
        print("testua: {}".format(line))
        print("tokenak: {}".format(token_line))
        print("---------------------------------------------------------")
        
  return tokens

def sentencepiece_tokenize_corpus(filepath):
  tokens = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      tokens += tokenizer.tokenize(line)      
  return tokens


bert_tokens = bert_tokenize_corpus("./corpus/eu/2014wiki.eu.sent_splited")
sentencepiece_tokens = sentencepiece_tokenize_corpus("./corpus/eu/2014wiki.eu.sent_splited")

print("wordpiece tokens: {}".format(len(bert_tokens)))
print("sentencepiece tokens: {}".format(len(sentencepiece_tokens)))

print("[UNK] kop: {}".format(bert_tokens.count('[UNK]')))

testua: astronomia (grekerazko ἄστρον, astron; "argizagi, zeruko objektu" eta νόμος, nomos, "arau, lege" hitzetatik: "argizagien legea") zeruko objektu edo argizagiak (hala nola izarrak, planetak, kometak, galaxiak) eta lurraren atmosferatik kanpo gertatzen diren fenomenoak (hondoko erradiazio kosmikoa, esaterako) aztertzen dituen zientzia da.

tokenak: ['astronomia', '(', 'gre', '##ker', '##azko', '[UNK]', ',', 'astron', ';', '"', 'argizagi', ',', 'zeru', '##ko', 'objektu', '"', 'eta', '[UNK]', ',', 'no', '##mos', ',', '"', 'arau', ',', 'lege', '"', 'hitze', '##tatik', ':', '"', 'argizagie', '##n', 'legea', '"', ')', 'zeru', '##ko', 'objektu', 'edo', 'argizagi', '##ak', '(', 'hala', 'nola', 'izarrak', ',', 'planetak', ',', 'kom', '##etak', ',', 'galaxi', '##ak', ')', 'eta', 'lurraren', 'atmosfera', '##tik', 'kanpo', 'gertatzen', 'diren', 'fenomenoak', '(', 'hond', '##oko', 'erradi', '##azio', 'kosm', '##ikoa', ',', 'esaterako', ')', 'aztertzen', 'dituen', 'zientzia', 'da', '.']
------

In [0]:
!python bert/create_pretraining_data.py \
  --input_file=$GS/corpus/eu/2014wiki.eu.sent_splited \
  --output_file=$GS/wordpiece/pretraining.tf.data \
  --vocab_file=vocab.txt \
  --do_lower_case=True \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=666 \
  --do_whole_word_mask=True \
  #--dupe_factor=5

In [0]:
'''
!python src/run_pretraining.py \
  --config_file eu.congif.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=1000000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
'''

!python bert/run_pretraining.py \
  --input_file=$GS/wordpiece/pretraining.tf.data \ \
  --output_dir=$GS/wordpiece/model \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert_config.json \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=10000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  #--num_train_steps=1000000 \
  #--init_checkpoint=$GS/wordpiece/model \

### EN-EU

In [15]:
import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('./spModels/en-eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

Learnt vocab size: 44999
Sample tokens: ['set', '▁gmt', 'rma', 'slapp', 'function', '▁via', 'rs', 'steinsaltz', '▁padding', '▁substantial']


In [0]:
# hau errepasatu. adibidez @ ez dut ondo egiten... [UNK] agertzen da...
# adibidez, 

#import string
#def parse_sentencepiece_token(token):
#    if token.startswith("▁"):
#        return token[1:]
#    else:
#        if token in string.punctuation:
#            return token
#        else:
#            return "##" + token
          

def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token          

In [17]:
import string

bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
# puntuazio sinboloak gehitu 
ctrl_symbols_end = list(string.punctuation)
#ctrl_symbols = ["[UNK]"]
bert_vocab = ctrl_symbols + bert_vocab + ctrl_symbols_end

#bert_vocab += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]
print(len(bert_vocab))

45036


In [0]:
VOC_FNAME = "vocab-en_eu.txt"
with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [19]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
print(bert_tokenizer.tokenize("Nere kotxea aitonaren etxe alboan dago!, eta bere kolorea gorria da. ni@ni.eus erabili "))

['ne', '##re', 'ko', '##txea', 'ai', '##ton', '##aren', 'etxe', 'alb', '##oa', '##n', 'dago', '!', ',', 'eta', 'bere', 'kolore', '##a', 'gorria', 'da', '.', 'ni', '@', 'ni', '.', 'eu', '##s', 'erabili']


In [0]:
FILES = "./corpus/en-eu/2014wiki.eu.sent_splited,./corpus/en-eu/2019wiki-10k.en.sent_splited"

!python bert/create_pretraining_data.py \
  --input_file={FILES} \
  --output_file=$GS/wordpiece/pretraining-en_eu.tf.data \
  --vocab_file=vocab-en_eu.txt \
  --do_lower_case=True \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=666 \
  --do_whole_word_mask=True \
  #--dupe_factor=5

In [0]:
'''
!python src/run_pretraining.py \
  --config_file eu.congif.ini \
  --input_file=$GS/pretraining.tf.data \
  --output_dir=$GS/eu.gureBERT \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=1000000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
'''

!python bert/run_pretraining.py \
  --input_file=$GS/wordpiece/pretraining-en_eu.tf.data \ \
  --output_dir=$GS/wordpiece/model-en_eu \
  --do_train=True \
  --do_eval=True \
  --bert_config_file=bert_config.json \
  --train_batch_size=64 \
  --max_seq_length=512 \
  --max_predictions_per_seq=20 \
  --num_train_steps=20000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4 \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --init_checkpoint=$GS/wordpiece/model-en_eu/ \
  #--num_train_steps=1000000 \
  

### SQuAD

In [0]:
# data English da!! baino kodea badabil 

!python bert/run_squad.py \
  --vocab_file=vocab-en_eu.txt \
  --bert_config_file=bert_config.json \
  --do_lower_case=True \
  --do_train=True \
  --train_file=train-v2.0.json \
  --do_predict=True \
  --predict_file=dev-v2.0.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=0.1 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --output_dir=gs://gurebert/gureBERT/wordpiece/squad/ \
  --use_tpu=True \
  --tpu_name=$TPU_NAME \
  --version_2_with_negative=True \
  --init_checkpoint=gs://gurebert/gureBERT/wordpiece/model-en_eu/ \

# gureBERT (japanese) erabiliz

In [0]:
#################################################################
# gureBERT japanese

!cd /content/

!git clone --recursive https://github.com/zmwebdev/bert-japanese
%cd bert-japanese



In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!python3 src/data-download-and-extract.py
!bash src/file-preprocessing.sh

#!gsutil cp -r gs://gurebert/gureBERT/data/wiki/AA/wiki_00 data/wiki/AA/wiki_00

# Oharra: esaldiak lerro bakarrean jarri behar dira
#!cat data/wiki/AA/wiki_00

In [0]:
# wiki-eu.vocab eta wiki-eu.model sortzen ditu

!pip install sentencepiece
!python3 src/train-sentencepiece.py

In [0]:
!gsutil cp -r data/ gs://gurebert/gureBERT/
!gsutil cp -r model/ gs://gurebert/gureBERT/

In [0]:
!mkdir model
!gsutil cp gs://gurebert/gureBERT/model/wiki-eu.vocab model
!gsutil cp gs://gurebert/gureBERT/model/wiki-eu.model model
!head -n 50 model/wiki-eu.vocab

<unk>	0
<s>	0
</s>	0
[PAD]	0
[CLS]	0
[SEP]	0
[MASK]	0
,	-2.87014
.	-2.98912
▁eta	-3.5406
▁ziren	-4.15683
-	-4.26943
an	-4.4068
▁da	-4.40907
a	-4.47601
▁zen	-4.51427
ko	-4.52925
▁	-4.67755
▁zeuden	-4.76555
▁bat	-4.84966
▁"	-4.95145
"	-4.95743
▁1	-5.04005
ak	-5.2203
▁izan	-5.32102
▁zuen	-5.34175
en	-5.36722
ren	-5.38606
:	-5.53342
k	-5.56319
▁(	-5.58916
▁zituzten	-5.58963
▁zuten	-5.59859
▁etxek	-5.66741
)	-5.66971
▁du	-5.70751
▁zituen	-5.72891
▁enpresak	-5.73378
▁bere	-5.7694
▁2	-5.78944
▁dago	-5.79301
▁2009	-5.8295
▁ere	-5.84878
▁2007	-5.87422
n	-5.87535
▁pertsona	-5.88342
▁biztanle	-5.93646
aren	-5.97756
▁dira	-5.97904
▁ditu	-6.0186


In [0]:
#https://github.com/zmwebdev/bert-japanese/blob/master/notebook/check-trained-tokenizer.ipynb
!pip install sentencepiece

import sys
sys.path.append("src")

import tokenization_sentencepiece as tokenization

text1 = "Nere kotxea aitonaren etxe alboan dago"
text2 = "Gorria da gure etxearen inguruan dagoen lorearen kolorea"

tokenizer = tokenization.FullTokenizer(
    model_file="model/wiki-eu.model",
    vocab_file="model/wiki-eu.vocab",
    do_lower_case=True)

tokenizer.tokenize(text1)

Loaded a trained SentencePiece model.


['▁nere', '▁kotxe', 'a', '▁aitona', 'ren', '▁etxe', '▁alboan', '▁dago']

In [0]:
tokenizer.tokenize(text2)

['▁gorria',
 '▁da',
 '▁gure',
 '▁etxearen',
 '▁inguruan',
 '▁dagoen',
 '▁lore',
 'aren',
 '▁kolorea']

In [0]:
#do_lower_case=False jarrita tokenizazioa okerragoa da
tokenizer = tokenization.FullTokenizer(
    model_file="model/wiki-eu.model",
    vocab_file="model/wiki-eu.vocab",
    do_lower_case=False)

tokenizer.tokenize(text1)

['▁', 'N', 'ere', '▁kotxe', 'a', '▁aitona', 'ren', '▁etxe', '▁alboan', '▁dago']

In [0]:
tokenizer.tokenize(text2)

['▁',
 'G',
 'orri',
 'a',
 '▁da',
 '▁gure',
 '▁etxearen',
 '▁inguruan',
 '▁dagoen',
 '▁lore',
 'aren',
 '▁kolorea']

In [0]:
# convert to WordPiece (kodean errorea dago). begiratu https://github.com/kwonmha/bert-vocab-builder/pull/4#issue-291306156

!git clone https://github.com/kwonmha/bert-vocab-builder.git

!python bert-vocab-builder/subword_builder.py \
--corpus_filepattern model/wiki-eu.vocab \
--output_filename model/wiki-eu-wordpiece.vocab
#--min_count {minimum_subtoken_counts}

# https://github.com/kwonmha/bert-vocab-builder/pull/4#issue-291306156

fatal: destination path 'bert-vocab-builder' already exists and is not an empty directory.
Traceback (most recent call last):
  File "bert-vocab-builder/subword_builder.py", line 33, in <module>
    import text_encoder
  File "/content/bert-japanese/bert-vocab-builder/text_encoder.py", line 647
    <<<<<<< HEAD
     ^
SyntaxError: invalid syntax


In [0]:
#Creating data for pretraining
#Create .tfrecord files for pretraining. For longer sentence data, replace the value of max_seq_length with 512.
!cat creating_data_for_pretraining.sh
!bash creating_data_for_pretraining.sh

for DIR in $( find data/wiki/ -mindepth 1 -type d ); do 
  python3 src/create_pretraining_data.py \
    --input_file=${DIR}/all.txt \
    --output_file=${DIR}/all-maxseq128.tfrecord \
    --model_file=./model/wiki-eu.model \
    --vocab_file=./model/wiki-eu.vocab \
    --do_lower_case=True \
    --max_seq_length=128 \
    --max_predictions_per_seq=20 \
    --masked_lm_prob=0.15 \
    --random_seed=12345 \
    --dupe_factor=5
done

In [0]:
# copy to gs
!gsutil cp -r data gs://gurebert/gureBERT
!gsutil cp -r model gs://gurebert/gureBERT
  

In [0]:
# Pre-Training
# https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/pretraining.ipynb

import tensorflow as tf

from google.colab import auth
auth.authenticate_user()

#Check TPU devices

import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
  


TPU address is grpc://10.16.115.170:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 5460104680638402674),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 16051411686286296001),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1933247979704415977),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 11046566402910946519),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 13421296677076221120),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 5352660506872180047),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 7874427115731877787),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 15986719012258325089),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 76127872665

In [0]:
INPUT_DATA_GCS = 'gs://gurebert/gureBERT/data/wiki'
TARGET_DIRS = [
  'AA',
  'AB',
  'AC'
]

MAX_SEQ_LEN = 128
#MAX_SEQ_LEN = 512


INPUT_FILE = ','.join( [ '{}/{}/all-maxseq{}.tfrecord'.format(INPUT_DATA_GCS, elem, MAX_SEQ_LEN) for elem in TARGET_DIRS] )

OUTPUT_GCS = 'gs://gurebert/gureBERT/model'

In [0]:
# Adding whole word masking aldaketa egin dute BERT-en. update egin behar da!!!
# Begiratu https://github.com/google-research/bert/commit/0fce551b55caabcfba52c61e18f34b541aef186a

!python src/run_pretraining.py \
  --input_file={INPUT_FILE} \
  --output_dir={OUTPUT_GCS} \
  --use_tpu=True \
  --tpu_name={TPU_ADDRESS} \
  --num_tpu_cores=8 \
  --do_train=True \
  --do_eval=True \
  --train_batch_size=64 \
  --max_seq_length={MAX_SEQ_LEN} \
  --max_predictions_per_seq=20 \
  --num_train_steps=1400000 \
  --num_warmup_steps=10000 \
  --save_checkpoints_steps=10000 \
  --learning_rate=1e-4

W0625 19:38:15.208214 140282338469760 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0625 19:38:15.209448 140282338469760 deprecation_wrapper.py:119] From src/run_pretraining.py:497: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0625 19:38:15.210220 140282338469760 deprecation_wrapper.py:119] From src/run_pretraining.py:412: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0625 19:38:15.210398 140282338469760 deprecation_wrapper.py:119] From src/run_pretraining.py:412: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0625 19:38:15.210564 140282338469760 deprecation_wrapper.py:119] From /content/bert-japanese/src/../bert/modeling.py:92: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0625 19:38:15.

In [0]:
##########################################
##########################################

# BERT-ekin konpatiblea den vocab sortzeko prozedura. wordpiece _ -> ##. 
# Honen abantaila BERT kodearen aldaketak eta run_squad.py, ... ezer aldatu gabe ibiliko dela da.
# berez ez da wordpiece, sentencepiece baizik baina sintaktikoki konpatiblea
# ikusi: https://colab.research.google.com/drive/1-uLyGTnz2K4gx3su6Qc00in9QCa4b5Kh#scrollTo=9S4CiOh3RzFW


In [0]:
import random

def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format('model/wiki-eu'))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

Learnt vocab size: 31999
Sample tokens: ['.036', '▁ukuilu', 'lok', '▁ahaltsua', '▁batzuei', '▁erabili', '▁chapman', '▁tv', '▁pe', '▁polemika']


In [0]:
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token

In [0]:
bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))

ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
bert_vocab = ctrl_symbols + bert_vocab

bert_vocab += ["[UNUSED_{}]".format(i) for i in range(32000 - len(bert_vocab))]
print(len(bert_vocab))

32004


In [0]:
VOC_FNAME = "vocab.txt" #@param {type:"string"}

with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [0]:
from bert import tokenization

bert_tokenizer = tokenization.FullTokenizer(VOC_FNAME)
bert_tokenizer.tokenize("Nere kotxea aitonaren etxe alboan dago")



['nere', 'kotxe', '##a', 'aitona', '##ren', 'etxe', 'alboan', 'dago']

In [0]:
#!gsutil cp -r gs://gurebert/gureBERT/data/wiki/AA/wiki_00 wiki_00
# ez du wiki-eu.model behar, vocab.txt-ekin nahikoa da

!python bert/create_pretraining_data.py \
  --input_file=wiki_00 \
  --output_file=/tmp/tf_examples.tfrecord \
  --vocab_file=vocab.txt \
  --do_lower_case=True \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --masked_lm_prob=0.15 \
  --random_seed=12345 \
  --dupe_factor=5

#############################################################
#############################################################

tf_examples.tfrecord  tmpcqwLhN
